In [ ]:
!pip install qiskit
!pip install qiskit-aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 105.0 MB/s eta 0:00:00


In [ ]:
import numpy as np
from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

# --- Helper Functions ---
def array_to_int(bit_array):
    """Converts a list of bits (0 or 1) to an integer."""
    out = 0
    for bit in bit_array:
        out = (out << 1) | bit
    return out

def int_to_bits(value, num_bits):
    """Converts an integer to a list of bits of specified length."""
    return [int(bit) for bit in bin(value)[2:].zfill(num_bits)]

# --- Binary Image Negation ---
def negate_binary_image_quantum(binary_image_flat):
    """
    Performs image negation on a flattened binary image using Qiskit.
    0 is considered black, 1 is considered white.
    Negation: 0 -> 1, 1 -> 0.

    Args:
        binary_image_flat (list of int): A flattened list of 0s and 1s representing the image.

    Returns:
        list of int: The negated flattened binary image.
        matplotlib.figure.Figure: Figure object for the histogram of results.
    """
    num_pixels = len(binary_image_flat)
    if num_pixels == 0:
        return [], None

    # Create a quantum circuit for the original input
    qr = QuantumRegister(num_pixels, "pixel")
    cr = ClassicalRegister(num_pixels, "measurement")
    qc_original = QuantumCircuit(qr, cr)

    # 1. Initialize qubits to represent the original input image
    # If pixel is 1, apply an X gate to flip the qubit from |0> to |1>
    for i in range(num_pixels):
        if binary_image_flat[i] == 1:
            qc_original.x(qr[i])

    qc_original.barrier()  # For visual separation

    print("--- Quantum Circuit for Original Binary Input ---")
    print(qc_original.draw(output="text"))

    # Create the full circuit for negation
    qc = QuantumCircuit(qr, cr)

    # 2. Initialize qubits to represent the input image (same as above)
    for i in range(num_pixels):
        if binary_image_flat[i] == 1:
            qc.x(qr[i])

    qc.barrier()  # For visual separation

    # 3. Apply X gate to all qubits to perform negation
    for i in range(num_pixels):
        qc.x(qr[i])

    qc.barrier()  # For visual separation

    # 4. Measure the qubits
    qc.measure(qr, cr)

    print("--- Binary Image Negation Quantum Circuit ---")
    print(qc.draw(output="text"))

    # Simulate the circuit
    backend = AerSimulator()
    transpiled_qc = transpile(qc, backend)
    job = backend.run(transpiled_qc, shots=10240)  # More shots for better statistics
    result = job.result()
    counts = result.get_counts(transpiled_qc)

    # Determine the most frequent result
    most_frequent_state_str = max(counts, key=counts.get)
    negated_image_flat = [int(bit) for bit in most_frequent_state_str][::-1]  # Reverse to match input order

    # Plot histogram
    hist_fig = plot_histogram(counts)

    return negated_image_flat, hist_fig

# --- Main Execution Example ---
if __name__ == "__main__":
    print("Make sure to install necessary packages in Colab: !pip install qiskit qiskit-aer matplotlib")

    # Example: Binary Image [0, 1, 1, 0] (could be a 2x2 image flattened)
    # Let 0 = black, 1 = white
    # Original: B W
    #           W B
    # Negated:  W B
    #           B W  => [1, 0, 0, 1]
    binary_image = [1, 0, 1, 0]
    print(f"\nOriginal Binary Image: {binary_image}")
    negated_binary, binary_hist_fig = negate_binary_image_quantum(binary_image)
    print(f"Negated Binary Image (Quantum): {negated_binary}")
    if binary_hist_fig:
        binary_hist_fig.suptitle("Binary Image Negation Measurement Counts")
        binary_hist_fig.savefig("binary_negation_histogram.png")
        print("Saved binary negation histogram to binary_negation_histogram.png")

Make sure to install necessary packages in Colab: !pip install qiskit qiskit-aer matplotlib

Original Binary Image: [1, 0, 1, 0]
--- Quantum Circuit for Original Binary Input ---
               ┌───┐ ░ 
      pixel_0: ┤ X ├─░─
               └───┘ ░ 
      pixel_1: ──────░─
               ┌───┐ ░ 
      pixel_2: ┤ X ├─░─
               └───┘ ░ 
      pixel_3: ──────░─
                     ░ 
measurement: 4/════════
                       
--- Binary Image Negation Quantum Circuit ---
               ┌───┐ ░ ┌───┐ ░ ┌─┐         
      pixel_0: ┤ X ├─░─┤ X ├─░─┤M├─────────
               └───┘ ░ ├───┤ ░ └╥┘┌─┐      
      pixel_1: ──────░─┤ X ├─░──╫─┤M├──────
               ┌───┐ ░ ├───┤ ░  ║ └╥┘┌─┐   
      pixel_2: ┤ X ├─░─┤ X ├─░──╫──╫─┤M├───
               └───┘ ░ ├───┤ ░  ║  ║ └╥┘┌─┐
      pixel_3: ──────░─┤ X ├─░──╫──╫──╫─┤M├
                     ░ └───┘ ░  ║  ║  ║ └╥┘
measurement: 4/═════════════════╩══╩══╩══╩═
                                0  1  2  3 
Negated Binary Image (Quant

In [ ]:
import numpy as np
from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector  # Added for statevector visualization
import matplotlib.pyplot as plt

# --- Grayscale Image Negation (Pixel by Pixel Quantum Bit-Flip) ---
def negate_grayscale_pixel_quantum(pixel_value, intensity_bits=8, print_details=False, pixel_coords_for_print="Sample Grayscale Pixel"):
    """
    Negates a single grayscale pixel value using a quantum circuit.
    Optionally prints circuit and statevectors before/after negation.
    """
    if not (0 <= pixel_value < (1 << intensity_bits)):
        raise ValueError(f"Pixel value {pixel_value} is out of range for {intensity_bits} bits.")

    qr = QuantumRegister(intensity_bits, "intensity")
    cr = ClassicalRegister(intensity_bits, "measurement")
    qc = QuantumCircuit(qr, cr)

    # 1. Encode the pixel_value
    binary_representation = bin(pixel_value)[2:].zfill(intensity_bits)
    for j in range(intensity_bits):
        if binary_representation[intensity_bits - 1 - j] == '1':  # Qiskit's LSB is q0
            qc.x(qr[j])

    qc.barrier(label="Initial_State")
    qc.save_statevector(label="state_before_negation")

    # 2. Apply X gate to all intensity qubits for negation
    for j in range(intensity_bits):
        qc.x(qr[j])

    qc.barrier(label="After_Negation")
    qc.save_statevector(label="state_after_negation")

    # 3. Measure
    qc.measure(qr, cr)

    if print_details:
        print(f"\n--- Quantum Details for {pixel_coords_for_print} (Input Value: {pixel_value}) ---")
        print("Quantum Circuit:")
        print(qc.draw(output="text"))

    backend = AerSimulator()  # AerSimulator supports save_statevector
    transpiled_qc = transpile(qc, backend)
    job = backend.run(transpiled_qc)  # shots=1 implied by save_statevector focus
    result = job.result()
    data = result.data(transpiled_qc)  # Or result.data() if transpiled_qc not needed for key

    if print_details:
        sv_before_data = data.get('state_before_negation')
        sv_after_data = data.get('state_after_negation')

        if sv_before_data is not None:
            print("\nStatevector BEFORE negation:")
            s_vec_obj_before = Statevector(sv_before_data)
            for i, amp in enumerate(s_vec_obj_before.data):
                if not np.isclose(abs(amp), 0.0):
                    print(f"{amp.real:.2f}{'+' if amp.imag>=0 else ''}{amp.imag:.2f}j |{bin(i)[2:].zfill(intensity_bits)}>")
            print(f"(Corresponds to classical value: {pixel_value})")
        else:
            print("\nStatevector BEFORE negation: Not found in simulation results.")

        if sv_after_data is not None:
            print("\nStatevector AFTER negation:")
            s_vec_obj_after = Statevector(sv_after_data)
            for i, amp in enumerate(s_vec_obj_after.data):
                if not np.isclose(abs(amp), 0.0):
                    print(f"{amp.real:.2f}{'+' if amp.imag>=0 else ''}{amp.imag:.2f}j |{bin(i)[2:].zfill(intensity_bits)}>")
        else:
            print("\nStatevector AFTER negation: Not found in simulation results.")

    counts = result.get_counts(transpiled_qc)
    measured_bitstring = list(counts.keys())[0]

    negated_value = 0
    # Qiskit's bitstring is q(N-1)...q(0). Classical int conversion usually reads MSB first.
    # The measured_bitstring is already in the correct order for int(measured_bitstring, 2)
    negated_value = int(measured_bitstring, 2)

    if print_details and sv_after_data is not None:
        print(f"(Corresponds to classical negated value: {negated_value})")
    elif print_details:
        print(f"Classical negated value from measurement: {negated_value}")

    return negated_value

def negate_grayscale_image_quantum(image_matrix, intensity_bits=8):
    """
    Performs quantum negation on a grayscale image, pixel by pixel.
    Modified to print circuits for all pixels.
    """
    if not image_matrix or not image_matrix[0]:
        return []
    height = len(image_matrix)
    width = len(image_matrix[0])
    negated_image_matrix = [[0 for _ in range(width)] for _ in range(height)]
    print(f"\n--- Negating Grayscale Image ({height}x{width}, {intensity_bits}-bit) Quantumly (Pixel by Pixel) ---")
    for r in range(height):
        for c in range(width):
            original_pixel = image_matrix[r][c]
            coords_str = f"Grayscale Pixel ({r},{c})"
            negated_pixel = negate_grayscale_pixel_quantum(original_pixel, intensity_bits, print_details=True, pixel_coords_for_print=coords_str)
            negated_image_matrix[r][c] = negated_pixel
    return negated_image_matrix

# --- Main Execution Example (Grayscale Only) ---
if __name__ == "__main__":
    print("Make sure to install necessary packages in Colab: !pip install qiskit qiskit-aer matplotlib")

    # --- Grayscale Image Example ---
    grayscale_image_example_3bit = [
        [1, 6],  # 1 (001) -> 6 (110), 6 (110) -> 1 (001)
        [3, 5]   # 3 (011) -> 4 (100), 5 (101) -> 2 (010)
    ]
    intensity_bits_gray = 3  # Using 3 bits for smaller statevector printout (0-7 range)

    print(f"\nOriginal Grayscale Image ({intensity_bits_gray}-bit):")
    for row in grayscale_image_example_3bit:
        print(row)

    negated_grayscale_example = negate_grayscale_image_quantum(grayscale_image_example_3bit, intensity_bits_gray)
    print(f"\nNegated Grayscale Image (Quantum, {intensity_bits_gray}-bit):")
    for row in negated_grayscale_example:
        print(row)

    print("\nNote: The quantum negation performed here is a bit-wise flip of the intensity values.")
    print("This is a pixel-by-pixel approach. For full Quantum Image Representations (QIRs) like NEQR or FRQI,")
    print("which encode spatial information for the entire image into a single quantum state, the circuits would be different and more complex.")

Make sure to install necessary packages in Colab: !pip install qiskit qiskit-aer matplotlib

Original Grayscale Image (3-bit):
[1, 6]
[3, 5]

--- Negating Grayscale Image (2x2, 3-bit) Quantumly (Pixel by Pixel) ---

--- Quantum Details for Grayscale Pixel (0,0) (Input Value: 1) ---
Quantum Circuit:
               ┌───┐ Initial_State  state_before_negation ┌───┐ After_Negation »
  intensity_0: ┤ X ├───────░──────────────────░───────────┤ X ├───────░────────»
               └───┘       ░                  ░           ├───┤       ░        »
  intensity_1: ────────────░──────────────────░───────────┤ X ├───────░────────»
                           ░                  ░           ├───┤       ░        »
  intensity_2: ────────────░──────────────────░───────────┤ X ├───────░────────»
                           ░                  ░           └───┘       ░        »
measurement: 3/════════════════════════════════════════════════════════════════»
                                                    

In [ ]:
import numpy as np
from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector  # Added for statevector visualization
import matplotlib.pyplot as plt

# --- Grayscale Pixel Negation (Used for Each Color Channel) ---
def negate_grayscale_pixel_quantum(pixel_value, intensity_bits=8, print_details=False, pixel_coords_for_print="Sample Grayscale Pixel"):
    """
    Negates a single grayscale pixel value using a quantum circuit.
    Optionally prints circuit and statevectors before/after negation.
    """
    if not (0 <= pixel_value < (1 << intensity_bits)):
        raise ValueError(f"Pixel value {pixel_value} is out of range for {intensity_bits} bits.")

    qr = QuantumRegister(intensity_bits, "intensity")
    cr = ClassicalRegister(intensity_bits, "measurement")
    qc = QuantumCircuit(qr, cr)

    # 1. Encode the pixel_value
    binary_representation = bin(pixel_value)[2:].zfill(intensity_bits)
    for j in range(intensity_bits):
        if binary_representation[intensity_bits - 1 - j] == '1':  # Qiskit's LSB is q0
            qc.x(qr[j])

    qc.barrier(label="Initial_State")
    qc.save_statevector(label="state_before_negation")

    # 2. Apply X gate to all intensity qubits for negation
    for j in range(intensity_bits):
        qc.x(qr[j])

    qc.barrier(label="After_Negation")
    qc.save_statevector(label="state_after_negation")

    # 3. Measure
    qc.measure(qr, cr)

    if print_details:
        print(f"\n--- Quantum Details for {pixel_coords_for_print} (Input Value: {pixel_value}) ---")
        print("Quantum Circuit:")
        print(qc.draw(output="text"))

    backend = AerSimulator()  # AerSimulator supports save_statevector
    transpiled_qc = transpile(qc, backend)
    job = backend.run(transpiled_qc)  # shots=1 implied by save_statevector focus
    result = job.result()
    data = result.data(transpiled_qc)

    if print_details:
        sv_before_data = data.get('state_before_negation')
        sv_after_data = data.get('state_after_negation')

        if sv_before_data is not None:
            print("\nStatevector BEFORE negation:")
            s_vec_obj_before = Statevector(sv_before_data)
            for i, amp in enumerate(s_vec_obj_before.data):
                if not np.isclose(abs(amp), 0.0):
                    print(f"{amp.real:.2f}{'+' if amp.imag>=0 else ''}{amp.imag:.2f}j |{bin(i)[2:].zfill(intensity_bits)}>")
            print(f"(Corresponds to classical value: {pixel_value})")
        else:
            print("\nStatevector BEFORE negation: Not found in simulation results.")

        if sv_after_data is not None:
            print("\nStatevector AFTER negation:")
            s_vec_obj_after = Statevector(sv_after_data)
            for i, amp in enumerate(s_vec_obj_after.data):
                if not np.isclose(abs(amp), 0.0):
                    print(f"{amp.real:.2f}{'+' if amp.imag>=0 else ''}{amp.imag:.2f}j |{bin(i)[2:].zfill(intensity_bits)}>")
        else:
            print("\nStatevector AFTER negation: Not found in simulation results.")

    counts = result.get_counts(transpiled_qc)
    measured_bitstring = list(counts.keys())[0]

    negated_value = int(measured_bitstring, 2)

    if print_details and sv_after_data is not None:
        print(f"(Corresponds to classical negated value: {negated_value})")
    elif print_details:
        print(f"Classical negated value from measurement: {negated_value}")

    return negated_value

# --- Color Image Negation (Pixel by Pixel Quantum Bit-Flip for Each Channel) ---
def negate_color_pixel_quantum(r_value, g_value, b_value, channel_bits=8, print_details=False, pixel_coords_for_print="Sample Color Pixel"):
    """
    Negates a single color pixel (R, G, B values) using quantum circuits for each channel.
    Optionally prints details for each channel.
    """
    if print_details:
        print(f"\n--- Processing {pixel_coords_for_print} (Input R:{r_value}, G:{g_value}, B:{b_value}) ---")

    negated_r = negate_grayscale_pixel_quantum(r_value, channel_bits, print_details=print_details,
                                               pixel_coords_for_print=f"{pixel_coords_for_print} - R Channel")
    negated_g = negate_grayscale_pixel_quantum(g_value, channel_bits, print_details=print_details,
                                               pixel_coords_for_print=f"{pixel_coords_for_print} - G Channel")
    negated_b = negate_grayscale_pixel_quantum(b_value, channel_bits, print_details=print_details,
                                               pixel_coords_for_print=f"{pixel_coords_for_print} - B Channel")
    return negated_r, negated_g, negated_b

def negate_color_image_quantum(image_matrix_rgb, channel_bits=8):
    """
    Performs quantum negation on a color image (RGB), pixel by pixel, channel by channel.
    Modified to print circuits for all pixels.
    """
    if not image_matrix_rgb or not image_matrix_rgb[0] or not image_matrix_rgb[0][0]:
        return []
    height = len(image_matrix_rgb)
    width = len(image_matrix_rgb[0])
    negated_image_matrix_rgb = [[(0,0,0) for _ in range(width)] for _ in range(height)]
    print(f"\n--- Negating Color Image ({height}x{width}x3, {channel_bits}-bit per channel) Quantumly (Pixel by Pixel) ---")
    for r_idx in range(height):
        for c_idx in range(width):
            original_r, original_g, original_b = image_matrix_rgb[r_idx][c_idx]
            coords_str = f"Color Pixel ({r_idx},{c_idx})"
            negated_r, negated_g, negated_b = negate_color_pixel_quantum(
                original_r, original_g, original_b, channel_bits,
                print_details=True, pixel_coords_for_print=coords_str)
            negated_image_matrix_rgb[r_idx][c_idx] = (negated_r, negated_g, negated_b)
    return negated_image_matrix_rgb

# --- Main Execution Example (Color Only) ---
if __name__ == "__main__":
    print("Make sure to install necessary packages in Colab: !pip install qiskit qiskit-aer matplotlib")

    # --- Color Image Example ---
    channel_bits_color = 8  # Using 8 bits for R,G,B channels (0-255 range)
    color_image_example = [
        [(3, 0, 1), (0, 3, 2)],
        [(1, 1, 1), (2, 1, 0)]
    ]
    print(f"\nOriginal Color Image ({channel_bits_color}-bit per channel):")
    for row in color_image_example:
        print(row)

    negated_color_example = negate_color_image_quantum(color_image_example, channel_bits_color)
    print(f"\nNegated Color Image (Quantum, {channel_bits_color}-bit per channel):")
    for row in negated_color_example:
        print(row)

    print("\nNote: The quantum negation performed here is a bit-wise flip of the intensity values.")
    print("This is a pixel-by-pixel approach. For full Quantum Image Representations (QIRs) like NEQR or FRQI,")
    print("which encode spatial information for the entire image into a single quantum state, the circuits would be different and more complex.")

Make sure to install necessary packages in Colab: !pip install qiskit qiskit-aer matplotlib

Original Color Image (8-bit per channel):
[(3, 0, 1), (0, 3, 2)]
[(1, 1, 1), (2, 1, 0)]

--- Negating Color Image (2x2x3, 8-bit per channel) Quantumly (Pixel by Pixel) ---

--- Processing Color Pixel (0,0) (Input R:3, G:0, B:1) ---

--- Quantum Details for Color Pixel (0,0) - R Channel (Input Value: 3) ---
Quantum Circuit:
               ┌───┐ Initial_State  state_before_negation ┌───┐ After_Negation »
  intensity_0: ┤ X ├───────░──────────────────░───────────┤ X ├───────░────────»
               ├───┤       ░                  ░           ├───┤       ░        »
  intensity_1: ┤ X ├───────░──────────────────░───────────┤ X ├───────░────────»
               └───┘       ░                  ░           ├───┤       ░        »
  intensity_2: ────────────░──────────────────░───────────┤ X ├───────░────────»
                           ░                  ░           ├───┤       ░        »
  intensity_3: 